# God Pipeline

In [152]:
import sys
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#1. load environment variables and data

# load environment variables
load_dotenv()

#add working directory to sys path to execute utils/dataset.py
working_dir = os.environ.get("WORKING_DIRECTORY")
sys.path.insert(0, working_dir)
from utils.dataset import get_data 
df = get_data()
df.head(10)

Loading data from wines: 0it [00:00, ?it/s]Exception during reset or similar
Traceback (most recent call last):
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/pool/base.py", line 991, in _finalize_fairy
    fairy._reset(
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/pool/base.py", line 1440, in _reset
    pool._dialect.do_rollback(self)
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/engine/default.py", line 657, in do_rollback
    dbapi_connection.rollback()
MySQLdb.OperationalError: (2013, 'Lost connection to MySQL server during query')
Exception during reset or similar
Traceback (most recent call last):
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/pool/base.py", line 991, in _finalize_fairy
    fairy._reset(
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/pool/base.py", line 1440, in _reset
    pool._dialect.do_rol

,wine type,fixed acidity,volatile acidity,citric acid,residual sugar,magnesium,flavanoids,minerals,calcium,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,Pinot noir,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,98.0,0.99290,3.19,0.48,9.2,5
1,Merlot,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,68.0,0.99163,2.96,0.52,11.1,6
2,Chardonnay,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,123.0,0.98949,3.24,0.36,12.6,7
3,Merlot,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,253.0,1.00014,3.02,0.56,9.1,6
4,Merlot,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,150.0,0.99508,3.23,0.37,10.0,6
5,Merlot,7.3,0.34,0.30,1.3,22.403749,1044.95,289.523749,267.12,0.057,25.0,173.0,0.99480,3.26,0.51,9.1,6
6,Merlot,7.6,0.21,0.49,2.5,23.875866,888.61,133.545866,109.67,0.047,20.0,130.0,0.99178,3.15,0.48,11.1,5
7,Chardonnay,6.0,0.25,0.40,5.7,23.309699,1381.79,266.529699,243.22,0.052,56.0,152.0,0.99398,3.16,0.88,10.5,6
8,Cabernet Sauvignon,6.7,0.18,0.19,4.7,49.165745,1456.41,269.915745,220.75,0.046,57.0,161.0,0.99460,3.32,0.66,10.5,6
9,Gamay,7.7,0.28,0.39,8.9,54.450579,929.44,377.690579,323.24,0.036,8.0,117.0,0.99350,3.06,0.38,12.0,2


In [153]:
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=[np.number]).columns
#drop 'quality' from numerical features (its a series)
numerical_features = numerical_features.drop('quality')
label = pd.Series('quality')

## God Pipeline

In [243]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
#import FunctionTransformer

### Cleaning Pipeline

#### Label Outlier Detection

In [244]:
def outlier_detection_label(df):
    #detect outliers and impute them with the simple imputer
    from sklearn.impute import KNNImputer

    #detect outliers with z-score and set them to NaN with = np.nan
    from scipy import stats
    z = np.abs(stats.zscore(df.iloc[:, df.shape[1]-1]))
    df.iloc[:, df.shape[1]-1][(z >= 3)] = np.nan


    #impute outliers with linear regression
    imputer = KNNImputer(n_neighbors=5).set_output(transform="pandas")
    df = imputer.fit_transform(df)
    
    return df

outlier_detection_label = FunctionTransformer(outlier_detection_label).set_output(transform="pandas")


/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


#### Feature Outlier Detection

In [ ]:
def outlier_detection(df):
    #detect outliers and impute them with the simple imputer
    from sklearn.impute import SimpleImputer

    #detect outliers
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1

    #detect outliers for each column and set them to NaN
    for col in df.columns:
        df.loc[(df[col] < (Q1[col] - 1.5 * IQR[col])) | (df[col] > (Q3[col] + 1.5 * IQR[col])), col] = np.nan

    #impute outliers with median
    imputer = SimpleImputer(strategy='median').set_output(transform="pandas")
    df = imputer.fit_transform(df)

    return df
outlier_detection = FunctionTransformer(outlier_detection).set_output(transform="pandas")

#### Feature Selection

In [292]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import json


def feature_selection(df, colinearity_threshold=0.5,correlation_threshold=0.1, vif_threshold=10.0):
    dropped_features = []  # List to store dropped features
    #-------------------------------------------------Cleaning-------------------------------------------------

    # exclude non numeric columns
    df = df.select_dtypes(exclude=['object'])
    
   #-------------------------------------------------Colinearity-------------------------------------------------
    # Calculate the correlation between columns
    corr_matrix = df.corr().abs()
    # Create a mask to select the upper triangle of the correlation matrix
    mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)

    # Apply the mask to get the upper triangle of the correlation matrix
    upper_triangle = corr_matrix.where(mask)
    
    # Find the columns with colinearity greater than correlation_threshold
    colinear_columns = upper_triangle[upper_triangle > colinearity_threshold].stack().index
     # Get the values of colinearity
    colinearity_values = upper_triangle[upper_triangle > colinearity_threshold].stack()

    # loop through the colinear_columns and select the one with lower correlation to quality and avoid duplicates in the list
    print("--------colinearity--------")
    for col1, col2 in colinear_columns:
        corr_col1 = df[col1].corr(df['label__quality'])
        corr_col2 = df[col2].corr(df['label__quality'])
        print("High Colinearity between " + col1 + " and "+col2 + " with a value of " + str(colinearity_values[col1, col2]))
        if abs(corr_col1) < abs(corr_col2):
                corr_col = df[col1].corr(df['label__quality'])
                if abs(corr_col) < correlation_threshold:
                    print("Dropping " + col1 + " because of low correlation "+str(corr_col1)+" with quality"+ "\n")
                    df = df.drop(col1, axis=1)
                    dropped_features.append(col1)
                else:
                    print("Not Dropping " + col1 + " because of high correlation "+str(corr_col1)+" with quality"+ "\n")

        else:
                corr_col = df[col2].corr(df['label__quality'])
                if abs(corr_col) < correlation_threshold:
                    print("Dropping " + col2 + " because of low correlation "+str(corr_col)+" with quality"+ "\n")
                    df = df.drop(col2, axis=1)
                    dropped_features.append(col2)
                else: 
                    print("Not Dropping " + col2 + " because of high correlation "+str(corr_col)+" with quality"+ "\n")
                     

    print("Every colinearity is below the threshold of "+str(colinearity_threshold)+"! \n")
    #-------------------------------------------------VIF-------------------------------------------------
    columns_to_exclude = ['label__quality']
    # Select the columns excluding the quality label
    columns = [col for col in df.select_dtypes(exclude='object').columns if col not in columns_to_exclude]

    # Create a new dataframe with only the selected columns
    df_selected = df[columns]

    # Add a constant column to the dataframe (required for VIF calculation)
    df_selected = sm.add_constant(df_selected)

    print("--------VIF--------"+ "\n")
    vifToRemove = 0
    while True:
        # Calculate VIF values for remaining features
        vif = pd.DataFrame()
        vif["Variable"] = df_selected.columns
        vif["VIF"] = [variance_inflation_factor(df_selected.values, i) if np.var(df_selected.iloc[:, i]) != 0 else 0 for i in range(df_selected.shape[1])]

        # Exclude the constant column from the results
        vif = vif[1:]
        # Order the VIF values in ascending order
        vif.sort_values('VIF', ascending=False, inplace=True)
        # Check if all VIF values are below the threshold
        if all(vif.iloc[vifToRemove:, vif.columns.get_loc("VIF")] < vif_threshold):
            print("Every VIF is below the threshold of "+str(vif_threshold)+"! \n")
            break

        # Find the feature with the highest VIF value
        if vifToRemove < vif.shape[0] and vif.iloc[vifToRemove]["Variable"] is not None:
            highest_vif_feature = vif.iloc[vifToRemove]["Variable"]
        
        # Check correlation of the highest VIF feature with quality label
        correlation = df[highest_vif_feature].corr(df["label__quality"])
        print("Highest VIF Value, Feature: " + str(highest_vif_feature) + " with a value of " + str(vif.iloc[vifToRemove]["VIF"]))
       

        if abs(correlation) < correlation_threshold:
            # Remove the feature if correlation is below the threshold
            print("Dropping " + highest_vif_feature + " because of low correlation "+str(correlation)+" with quality"+ "\n")
            dropped_features.append(highest_vif_feature)
            df = df.drop(highest_vif_feature, axis=1)
            df_selected = df_selected.drop(highest_vif_feature, axis=1)  # Drop from df_selected as well
        else:
            print("Not Dropping " + highest_vif_feature + " because of high correlation "+str(correlation)+" with quality"+ "\n")
            # Move on to the next highest VIF feature
            vifToRemove = vifToRemove + 1
    
    # Save dropped features list to a JSON file
    with open('dropped_features.json', 'w') as f:
        json.dump(dropped_features, f)

    return df
feature_selection = FunctionTransformer(feature_selection).set_output(transform="pandas")

/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


In [293]:
cleaning_pipeline = Pipeline(steps=[
])

categorical_imputer = SimpleImputer(strategy="most_frequent").set_output(transform="pandas")

numerical_imputer = SimpleImputer(strategy="mean").set_output(transform="pandas")

#pipeline for categorical features
categorical_pipeline = Pipeline(steps=[])
categorical_pipeline.steps.append(('imputer', categorical_imputer))
categorical_pipeline.steps.append(('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform="pandas")))

#pipeline for numerical features
numeric_pipeline = Pipeline(steps=[])
numeric_pipeline.steps.append(('imputer', numerical_imputer))
numeric_pipeline.steps.append(('outlier_detection', outlier_detection))

#pipeline for label
label_pipeline = Pipeline(steps=[])
label_pipeline.steps.append(('imputer', numerical_imputer))

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
        ('label', label_pipeline, label)
    ]).set_output(transform="pandas")
cleaning_pipeline.steps.append(('preprocessor', preprocessor))
cleaning_pipeline.steps.append(("outlier_detection_label", outlier_detection_label))
cleaning_pipeline.steps.append(('feature_selection', feature_selection))
cleaning_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('outlier_detection',
                                                                   FunctionTransformer(func=<function outlier_detection at 0x15095b240>))]),
                                                  Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'magnesium', 'flavanoids', 'minerals', 'calcium', 'chlorides',
       'free sulfur dioxid...
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['wine type'], dtype='object')),
                                                 ('label',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  0    quality
dtype: object)])),
                ('outlier_detection_label',
                 FunctionTransformer(func=<function outlier_detection_label at 0x14f6bf100>)),
                ('feature_selection',
                 FunctionTransformer(func=<function feature_selection at 0x158912020>))])

In [248]:
from sklearn.ensemble import RandomForestRegressor

training_pipeline_random_forst = Pipeline(
    steps=[("model", RandomForestRegressor())]
)
model = {
        "name": "RandomForestRegressor",
        "estimator": RandomForestRegressor(),
        "hyperparameters":
            {
                "model__n_estimators": [200],
                "model__criterion": ["squared_error"],
                "model__max_depth": [None],
                "model__min_samples_split": [2],
                "model__min_samples_leaf": [1]
            }
    }

In [250]:
#God Function based on example: LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

def god_function(dirty_df):
    
    clean_df = pd.DataFrame(cleaning_pipeline.fit_transform(dirty_df))
    X = clean_df.drop('label__quality', axis=1)
    y = clean_df['label__quality']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
    grid = GridSearchCV(training_pipeline_random_forst, model["hyperparameters"], cv=5)
    grid = grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    print(grid.score(X_test, y_test))

god_function(df)

{'model__criterion': 'squared_error', 'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
0.837346033208167
0.9143587566685193


In [296]:
clean_df = pd.DataFrame(cleaning_pipeline.fit_transform(df))
X = clean_df.drop('label__quality', axis=1)
y = clean_df['label__quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
best_model = RandomForestRegressor(criterion = 'squared_error', max_depth = None, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200)
best_model = best_model.fit(X_train, y_train)
print(best_model.score(X_test, y_test))

#export best_model with pickle
import pickle
pickle.dump(best_model, open('best_model__random_forest.pkl','wb'))

--------colinearity--------
High Colinearity between num__residual sugar and num__density with a value of 0.8316095689674599
Dropping num__residual sugar because of low correlation -0.07760006766406455 with quality

High Colinearity between num__minerals and num__calcium with a value of 0.9032394680532317
Dropping num__calcium because of low correlation -0.021263570937899844 with quality

High Colinearity between num__chlorides and num__alcohol with a value of 0.5298204778821368
Not Dropping num__chlorides because of high correlation -0.1936620103051993 with quality

High Colinearity between num__free sulfur dioxide and num__total sulfur dioxide with a value of 0.6048632677816177
Dropping num__free sulfur dioxide because of low correlation 0.019648020910944565 with quality

High Colinearity between num__total sulfur dioxide and num__density with a value of 0.5523519514518478
Not Dropping num__total sulfur dioxide because of high correlation -0.10790569243506681 with quality

High Colin

/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


0.9069243948295124


In [262]:
#load best model with pickle
best_model = pickle.load(open('best_model__random_forest.pkl','rb'))
#select randomly 1000 data points from df and drop selected ones
df_validation = df.sample(n=1000, random_state=1)
clean_df = df.drop(df_validation.index)

X = clean_df.drop('label__quality', axis=1)
y = clean_df['label__quality']

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=200, stratify=y)

print(best_model.score(X_validation, y_validation))

0.970388005746285


# Simulation Meth-Daten

In [300]:
df_validation = df.sample(n=1000, random_state=42)

clean_df = df.drop(df_validation.index)
clean_df = pd.DataFrame(cleaning_pipeline.fit_transform(clean_df))

X_clean = clean_df.drop('label__quality', axis=1)
y_clean = clean_df['label__quality']

X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(X_clean, y_clean, test_size=0.2, random_state=200, stratify=y_clean)
best_model = RandomForestRegressor(criterion = 'squared_error', max_depth = None, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 200)
best_model = best_model.fit(X_train_clean, y_train_clean)


--------colinearity--------
High Colinearity between num__residual sugar and num__density with a value of 0.830780406761989
Dropping num__residual sugar because of low correlation -0.08364822160050356 with quality

High Colinearity between num__minerals and num__calcium with a value of 0.9030737715248937
Dropping num__calcium because of low correlation -0.02806247904531664 with quality

High Colinearity between num__chlorides and num__alcohol with a value of 0.5286770193126731
Not Dropping num__chlorides because of high correlation -0.19340534323337397 with quality

High Colinearity between num__free sulfur dioxide and num__total sulfur dioxide with a value of 0.6009950373633854
Dropping num__free sulfur dioxide because of low correlation 0.013309191783902859 with quality

High Colinearity between num__total sulfur dioxide and num__density with a value of 0.5511630654263218
Not Dropping num__total sulfur dioxide because of high correlation -0.12018177495915403 with quality

High Coline

/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [301]:
best_model.score(X_test_clean, y_test_clean)

0.857862219133906

In [302]:
df_validation = pd.DataFrame(cleaning_pipeline.fit_transform(df_validation))
X_validation = df_validation.drop('label__quality', axis=1)
y_validation = df_validation['label__quality']

best_model.score(X_validation, y_validation)

--------colinearity--------
High Colinearity between num__residual sugar and num__density with a value of 0.83747432504066
Dropping num__residual sugar because of low correlation -0.03841622427373328 with quality

High Colinearity between num__minerals and num__calcium with a value of 0.8940828932837327
Dropping num__minerals because of low correlation 0.020731400128316195 with quality

High Colinearity between num__chlorides and num__alcohol with a value of 0.5384231056394151
Not Dropping num__chlorides because of high correlation -0.20146010910088719 with quality

High Colinearity between num__free sulfur dioxide and num__total sulfur dioxide with a value of 0.6284853559467134
Dropping num__total sulfur dioxide because of low correlation -0.02969884939978694 with quality



KeyError: 'num__total sulfur dioxide'